# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>

#### <center> **Final Project: Machine Learning** </center>
---

**Date**: November, 2025

**Student Name**: Luis Antonio Pelayo Sierra

**Professor**: Pablo Camarillo Ramirez

# Machine Learning algorithm to use

El problema que este algoritmo de machine learning intentara resolver es uno de clasificacion, en las dos entregas anteriores he experimentado nuevos datos que se pueden agregar al dataset original, uno de ellos siendo una clara clasificacion, que los divide en base a la popularidad del contenido. El objetivo es que en base a la mayoria de los datos, sin utilizar las metricas de likes y retweets, pueda predecir el tipo de impacto que un tweet tendra.

Para ello, se implementara un modelo de clasificacion basado en Random Forest, por que considero que por la naturaleza de los datos, es muy probable que exista overfitting, por lo que este algoritmo puede ayudar a disminuir el sobreajuste.

# Dataset Description

El dataset utilizado para esta actividad sera el Twitter-Dataset de Kaggle (https://www.kaggle.com/datasets/goyaladi/twitter-dataset/data), el cual consiste en informacion de 10,000 tweets del primer semestre de 2023, lo que incluye una gran variedad de usuarios, texto, cuentas de retweets y likes, ademas de los timestamps asociados a cada tweet. 

El dataset base contiene 6 columnas, ademas a estas, he decidido agregar nuevas columnas derivadas de la informacion base (similar a mis entregas pasadas, tanto las mismas como nuevas), con el fin de facilitar las predicciones, puesto que quitando los contadores de likes y retweets, habria muy pocas features relevantes.

In [8]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Machine Learning") \
    .master("spark://spark-master:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("INFO")

In [9]:
from luis_pelayo.spark_utils import SparkUtils

tweets_schema = SparkUtils.generate_schema([
    ("Tweet_ID", "int"),
    ("Username", "string"),
    ("Text", "string"),
    ("Retweets", "int"),
    ("Likes", "int"),
    ("Timestamp", "string")
])

tweets_df = spark.read \
    .option("header", "true") \
    .schema(tweets_schema) \
    .csv("/opt/spark/work-dir/data/machine_learning/twitter_dataset.csv")

tweets_df = tweets_df.na.drop()

In [10]:
from pyspark.sql.functions import col, when, size, length, split, to_timestamp, hour, dayofweek, month, year, dayofmonth

tweets_df = tweets_df.withColumn("Engagement", col("Likes") + col("Retweets"))
tweets_df = tweets_df.withColumn("Text_Length", length(col("Text")))
tweets_df = tweets_df.withColumn("Word_Count", size(split(col("Text"), " ")))

tweets_df = tweets_df.withColumn(
    "Avg_Word_Length",
    when(col("Word_Count") > 0, col("Text_Length") / col("Word_Count")).otherwise(0)
)

tweets_df = tweets_df.withColumn(
    "Hashtag_Count",
    size(split(col("Text"), "#")) - 1
)

tweets_df = tweets_df.withColumn("Timestamp_parsed", to_timestamp(col("Timestamp")))
tweets_df = tweets_df.withColumn("Hour", hour(col("Timestamp_parsed")))
tweets_df = tweets_df.withColumn("Day", dayofmonth(col("Timestamp_parsed")))
tweets_df = tweets_df.withColumn("Month", month(col("Timestamp_parsed")))
tweets_df = tweets_df.withColumn("Year", year(col("Timestamp_parsed")))
tweets_df = tweets_df.withColumn("DayOfWeek", dayofweek(col("Timestamp_parsed")))

tweets_df = tweets_df.withColumn(
    "Is_Weekend",
    when((col("DayOfWeek") == 1) | (col("DayOfWeek") == 7), 1).otherwise(0)
)

tweets_df = tweets_df.withColumn(
    "Time_Category",
    when((col("Hour") >= 6) & (col("Hour") < 12), 0) 
    .when((col("Hour") >= 12) & (col("Hour") < 18), 1)  
    .when((col("Hour") >= 18) & (col("Hour") < 22), 2) 
    .otherwise(3)  
)

# Label (antes era popularidad)
tweets_df = tweets_df.withColumn(
    "label",
    when(col("Engagement") >= 100, 3) # A mayor numero, mayor popularidad (nivel de popularidad)
    .when(col("Engagement") >= 50, 2)
    .when(col("Engagement") >= 10, 1)
    .otherwise(0)
)

# tweets_df.printSchema()
# tweets_df.show(5)

# ML Training process

In [11]:
# balance_counts = tweets_df.groupBy("label").count().orderBy("label")
# balance_counts.show()

In [12]:
from pyspark.ml.feature import VectorAssembler

feature_cols = [ "Text_Length", "Word_Count", "Avg_Word_Length", "Hashtag_Count", "Hour", 
                "Day", "Month", "DayOfWeek", "Is_Weekend", "Time_Category"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
assembled_df = assembler.transform(tweets_df)
assembled_df = assembled_df.select("features", "label")

train_df, test_df = assembled_df.randomSplit([0.8, 0.2], seed=57)

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

rf_model = RandomForestClassifier(
    labelCol="label",
    featuresCol="features",
    numTrees=200,
    maxDepth=20,
    seed=57
)

rf_trained = rf_model.fit(train_df)

25/11/25 00:12:33 WARN DAGScheduler: Broadcasting large task binary with size 1038.6 KiB
25/11/25 00:12:33 WARN DAGScheduler: Broadcasting large task binary with size 1764.0 KiB
25/11/25 00:12:34 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
25/11/25 00:12:35 WARN DAGScheduler: Broadcasting large task binary with size 4.4 MiB
25/11/25 00:12:37 WARN DAGScheduler: Broadcasting large task binary with size 6.6 MiB
25/11/25 00:12:38 WARN DAGScheduler: Broadcasting large task binary with size 1287.0 KiB
25/11/25 00:12:39 WARN DAGScheduler: Broadcasting large task binary with size 9.2 MiB
25/11/25 00:12:40 WARN DAGScheduler: Broadcasting large task binary with size 1600.0 KiB
25/11/25 00:12:41 WARN DAGScheduler: Broadcasting large task binary with size 12.3 MiB
25/11/25 00:12:42 WARN DAGScheduler: Broadcasting large task binary with size 1838.4 KiB
25/11/25 00:12:44 WARN DAGScheduler: Broadcasting large task binary with size 15.6 MiB
25/11/25 00:12:45 WARN DAGScheduler: 

# ML Evaluation

In [14]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

predictions = rf_trained.transform(test_df)

evaluator = MulticlassClassificationEvaluator(labelCol="label",
                            predictionCol="prediction")

f1 = evaluator.evaluate(predictions, 
            {evaluator.metricName: "f1"})
print(f"F1 Score: {f1:.4f}")

accuracy = evaluator.evaluate(predictions, 
            {evaluator.metricName: "accuracy"})
print(f"Accuracy: {accuracy:.4f}")

25/11/25 00:12:51 WARN DAGScheduler: Broadcasting large task binary with size 10.3 MiB
                                                                                

F1 Score: 0.4091


25/11/25 00:12:52 WARN DAGScheduler: Broadcasting large task binary with size 10.3 MiB
[Stage 39:>                                                         (0 + 1) / 1]

Accuracy: 0.4748


# Conclusiones

Se que para muchos, estos valores se podrian considerar un fracaso, intente reajustar el entrenamiento, modificar las propiedades del modelo, pero basicamente los resultados eran los mismos, pero hay una justificacion. La popularidad de un tweet no se basa unicamente en los datos del mismo, un factor primordial son los usuarios, tanto los que publican los tweets, como con los que interactuan mediante likes o respuestas. El modelo podria ser mas acertado si tuviera acceso a datos como los promedios historicos de popularidad del usuario que publica, pero siempre existiran variantes que impidan determinar cual va a ser el proximo "hit tweet".

In [ ]:
sc.stop()